In [5]:
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import pandas as pd
from crud_module import CRUD

# Initialize CRUD object with MongoDB connection
username = "aacuser"
password = "12345"
database = "AAC"
crud = CRUD(username, password, database)

# Retrieve data from MongoDB
data = crud.read({})

# Convert MongoDB data to DataFrame
df = pd.DataFrame.from_records(data)

# Drop '_id' column if it exists in the data
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Initialize Dash app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Img(src='/assets/grazioso-logo.png', style={'height': '100px'}),  # Grazioso Salvare Logo
    html.Center(html.B(html.H1('Grazioso Salvare Animal Rescue Dashboard'))),  # Dashboard title
    html.P("Developed by: Samuel Rincon"),  # Unique identifier

    # Dropdown filter for rescue type
    dcc.Dropdown(
        id='rescue-type-dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
        ],
        value='All',
        placeholder="Select Rescue Type"
    ),

    # Data table displaying the animal data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Display 10 rows at a time
        sort_action='native',
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'},  # Allow horizontal scrolling
    ),

    # Bar chart for breed counts, with added margin below
    html.Div(dcc.Graph(id='bar-chart'), style={'margin-bottom': '30px'}),  # Added margin between chart and map

    # Geolocation map to display animal location, with additional styling
    html.Div(id='map-id', className='col s12 m6', style={'width': '100%', 'height': '500px', 'margin-top': '20px'}),
])

# Callback to filter the data table based on rescue type selection
@app.callback(
    Output('datatable-id', 'data'),
    Input('rescue-type-dropdown', 'value')
)
def update_table(rescue_type):
    if rescue_type == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['rescue_type'] == rescue_type]
    return filtered_df.to_dict('records')

# Callback to update the geolocation map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Default map location set to Austin, TX if no data is selected
    if viewData is None or index is None or len(index) == 0:
        return [dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.2672, -97.7431], zoom=10, children=[dl.TileLayer()])]
    
    dff = pd.DataFrame.from_dict(viewData)  # Convert table data to DataFrame
    row = index[0]  # Get selected row index
    
    # Safely retrieve latitude and longitude, defaulting to Austin, TX if missing
    lat = dff.iloc[row].get("location_lat", None)
    lon = dff.iloc[row].get("location_long", None)
    
    if pd.isnull(lat) or pd.isnull(lon):  # Check for missing values
        lat, lon = 30.2672, -97.7431  # Default to Austin, TX
    
    # Return map with marker at the selected location
    return [dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(dff.iloc[row].get('breed', 'Unknown Breed')),
            dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row].get('animal_id', 'Unknown'))])
        ])
    ])]

# Callback to update the bar chart based on the filtered table data
@app.callback(
    Output('bar-chart', 'figure'),
    Input('datatable-id', 'derived_virtual_data')
)
def update_bar_chart(viewData):
    if viewData is None:
        return {}  # Return an empty chart if no data is available
    
    dff = pd.DataFrame.from_dict(viewData)  # Convert data to DataFrame
    breed_count = dff['breed'].value_counts()  # Count breeds
    
    # Create a bar chart of breed counts
    fig = {
        'data': [{'x': breed_count.index, 'y': breed_count.values, 'type': 'bar'}],
        'layout': {'title': 'Animal Count by Breed'}
    }
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


Connected to MongoDB successfully.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

